In [0]:
# usar version 2 de tensorflow en colab
!pip install tensorflow==2.0.0-alpha0
from __future__ import absolute_import, division, print_function

# W2 Augmentation

In [0]:
import os # necesario para usar rutas
import zipfile # necesario para extraer archivos
import random
import tensorflow as tf
from shutil import copyfile

# wget (para descargar archivos)
# forma: !wget #pagina# -O /tmp/destino
!wget "http://ruta.com/archivo.zip" -O "/tmp/cats-and-dogs.zip"

# Lo extraemos
# Establecemos los directorios base con ayuda de
# os.path.join, os.listdir y len(os.listdir)
len(os.listdir(dir)) # esto muestra el numero de archivos en cada carpeta

# Si hace falta, creamos las carpetas necesarias
os.mkdir('/tmp/carpeta')

Si tenemos un pool de archivos, separamos entre archivos de entrenamiento y de validación:

In [0]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    files = []
    for filename in os.listdir(SOURCE):
        file = SOURCE + filename
        if os.path.getsize(file) > 0:
            files.append(filename)
        else: # esto se hace para eliminar archivos erroneos sin tamaño
            print(filename + " is zero length, so ignoring.")

    training_length = int(len(files) * SPLIT_SIZE)
    testing_length = int(len(files) - training_length)
    shuffled_set = random.sample(files, len(files))
    training_set = shuffled_set[0:training_length]
    testing_set = shuffled_set[:testing_length]

    for filename in training_set:
        this_file = SOURCE + filename
        destination = TRAINING + filename
        copyfile(this_file, destination)

    for filename in testing_set:
        this_file = SOURCE + filename
        destination = TESTING + filename
        copyfile(this_file, destination)

### Otra forma de hacer esto sería (hecho por mi):

def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
  origin = os.listdir(SOURCE)
  num = len(origin)
  train_num = round(num*SPLIT_SIZE,0)
  test_num = round(num*(1-SPLIT_SIZE),0)
  shuffle = random.sample(origin, len(origin))
  train = shuffle[:int(train_num)]
  test = shuffle[int(train_num):]
  
  for i, image in enumerate(train):
    if os.path.getsize(os.path.join(SOURCE, image)) == 0:
      print('Image %s not copying. Zero filesize' % image)
    else:
      copyfile(os.path.join(SOURCE, image), os.path.join(TRAINING, image))
  
  for i, image in enumerate(test):
    if os.path.getsize(os.path.join(SOURCE, image)) == 0:
      print('Image %s not copying. Zero filesize' % image)
    else:
      copyfile(os.path.join(SOURCE, image), os.path.join(TESTING, image))
      
      
      
####

CAT_SOURCE_DIR = "/tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "/tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "/tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "/tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "/tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "/tmp/cats-v-dogs/testing/dogs/"

split_size = .9 # esto es el % que irá a cada carpeta
split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Para comprobar que se ha hecho bien, mostramos los archivos de cada carpeta:
print(len(os.listdir('/tmp/cats-v-dogs/training/cats/')))

In [0]:
# Comprobamos lss dimensiones de las imágenes
from tensorflow.keras.preprocessing.image import img_to_array, load_img
for i in range(10):
  img_dir = os.listdir(TRAINING_DOGS_DIR)
  img = load_img(os.path.join(TRAINING_DOGS_DIR, img_dir[i]))
  img_prueba = img_to_array(img)
  print(img_prueba.shape)

Ahora creamos el modelo, mostramos summary y lo compilamos:

In [0]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(... RECUERDA input_shape=(150, 150, 3)),
    ...
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

Ahora añadimos el generador. Para AUMENTAR LOS DATOS:


In [0]:
# https://keras.io/preprocessing/image/
TRAINING_DIR = "/tmp/cats-v-dogs/training/"
train_datagen = ImageDataGenerator(rescale=1./255,
      rotation_range=40, # rotación
      width_shift_range=0.2, # movimiento del centro hacia up-down
      height_shift_range=0.2,
      shear_range=0.2, # deformación elástica
      zoom_range=0.2, # zoom de imagen
      horizontal_flip=True, # espejo
      fill_mode='nearest') # como rellenar pixeles vacios
train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                                                    batch_size=100,
                                                    class_mode='binary',
                                                    target_size=(150, 150))

VALIDATION_DIR = "/tmp/cats-v-dogs/testing/"
validation_datagen = ImageDataGenerator(rescale=1./255,
      #generalmente aqui no se pone nada)
validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                                              batch_size=100,
                                                              class_mode='binary',
                                                              target_size=(150, 150))
                                        
# No siempre hay que hacer augmentation, porque si le damos demasiada
# variabilidad puede que no salga bien (overfitting). A veces es mejor nada
# sobre todo si el dataset no es muy completo.

Mostrar gráficos con matplotlib de training y validation:

In [0]:
%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

# Y el resto como se hace siempre

# W3 TRANSFER LEARNING

Primero descargamos el modelo a cargar con o sin los pesos:

In [0]:
# https://www.tensorflow.org/tutorials/images/transfer_learning
import os
from tensorflow.keras import layers
from tensorflow.keras import Model

# CARGAR UN MODELO
from tensorflow.keras.applications.inception_v3 import InceptionV3
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 'rgb obligatorio'
                                include_top = False, 'eliminar ultima capa'
                                weights = None) # o 'imagenet'

# CARGAR PESOS DESCARGADOS
wget http://inception_v3_weights_tf_dim_ordering_notop.h5 -O /tmp/modelo
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_notop.h5'
pre_trained_model.load_weights(local_weights_file)
  
# pre_trained_model.summary()

Obtener nombres para crear el modelo completo:

In [0]:
# OBTENER NOMBRE DE CAPA PARA 
last_layer = pre_trained_model.get_layer('mixed7')
# Debemos usar el nombre que veamos en el summary para la ultima capa
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

Para crear nuestro modelo completo, le añadimos las capas finales que queramos, y LO COMPILAMOS:

In [0]:
# Hay que hacerlo siguiendo la functional API

from tensorflow.keras.optimizers import RMSprop

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x) # ESTO AÑADE REGULARIZACIÓN, evita overfitting                 
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) # Model(inputs =, outputs =)

model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])


# Otra forma de hacer el modelo con la API secuencial:
model = tf.keras.Sequential([
  pre_trained_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(1, activation='sigmoid')
])

Toquetear el modelo y hacerlo entrenable:

In [0]:
# PARA HACERLO ENTRENABLE (TODAS LAS CAPAS)
for layer in pre_trained_model.layers:
  layer.trainable = False
  
# Otra forma: pre_trained_model.trainable = False

# VER VARIABLES ENTRENABLES:
len(pre_trained_model.trainable_variables)

# ENTRENAR SOLO CIERTAS CAPAS
# Vemos numero total de capas
print("Number of layers in the base model: ", len(base_model.layers))

# Elegimos una capa a partir de la que entrenar (o la buscamos)
fine_tune_at = 100 # a partir de la capa 100

# Congelamos todas las capas anteriores
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# IMAGEN DEL MOVELO
%matplotlib inline
from keras.utils import plot_model
plot_model(pre_trained_model, to_file='model.png')

from IPython.display import Image 
Image(filename='model.png')

# W4 MULTICLASS Clasification

In [0]:
# En vez de crear un subdirectorio en training-testing creamos 'x' directorios
# y lo mostramos
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_rock = [os.path.join(rock_dir, fname) 
                for fname in rock_files[pic_index-2:pic_index]]
next_paper = [os.path.join(paper_dir, fname) 
                for fname in paper_files[pic_index-2:pic_index]]
next_scissors = [os.path.join(scissors_dir, fname) 
                for fname in scissors_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_rock+next_paper+next_scissors):
  #print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

Ahora a la hora de crear el modelo tan solo hay que cambiar:

In [0]:
train_generator = training_datagen.flow_from_directory(
	...
	class_mode='categorical')

model = tf.keras.models.Sequential([
   ...
   tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# Cuando ya tenemos el modelo entrenado lo podemos guardar
model.save("saved_model.h5")